### Problem Set 7: Multitape Turing Machine

csc427: Theory of Automata and Complexity. 
<br>
university of miami
<br>
spring 2022.
<br>
Burton Rosenberg.
<br>
<br>
created: April 15, 2021
<br>last update: March 28, 2022


---

### Student name: Dominick Bello

---

### Overview

We will continue practive with turing machines by programming some more 
interesting problems using a multi-tape turing machine. The problem set
will also introduce Computable Functions (Sipser section 53, 3ird edition
page 234, 2nd edition page 206). 


The multitape turing machines will start with the input properly formated
on the main tape, with all other tapes empty. For the computable function,
it halts with accept and the answer written on the main tape. It can
also reject, for instance, if no output is possible because the input is 
not properly formatted.

It is possible that the turing machine does not halt, because the function
is not computable, similar to a set element problem not being decidable. 


#### Multitape syntax

The code for some of this project is in separate files, that you must copy to
this folder, alongside this file. The defines and classes will become available
to this code by import statements. 

The following imports brings in the TuringMachine and MachineParser classes.


In [1]:
from turing_machine_sim import *
from proj7_util import *


The MachineParser class takes a turing machine description and initializes a
TuringMachine object that can run the described program. The syntax of the
description was given in the previous problem set.

That description only described a single tape machine. For a multitape
machine the additions and emendations are the tapes stanza, and a generalization
of the StateTransition continuation line,

- __Tapes__ -> "start" ":" a positive integer
- __StateTransition__ -> (__Symbol__|__Special__)^k (__Symbol__|__Special__)^k __Action__^k __Ident__

The tape stanza must occur before any state stanza. 

The StateTransition line is interpreted as first the k tape symbols to match
on the k tapes; then the k tape symbols to write on the k tapes, then the k 
actions to undertake on the k tapes. 

If any action is capitalized, the configuration is printed. This allows for
targeted debugging.

The colon (:) is a wildcard. It's use among the k read symbols matches any symbol. 
Four use cases are allowed, and are listed in the priority the case is applied, 

- No wildcards. An exact match of the k type symbols has top precedence.
- One wildcard. An exact match for all but one symbol is tried if there is no exact match.
- k-1 wildcards. An exact match for exactly one symbol, the k-1 are wildcards, is tried next.
- All wildcards. The default matches anything.

When the wildcard appearing as a write symbol, it is set equal to the read symbol, on the 
corresponding tape.

If the machine rejects, it can be queried for the cause of the reject,

- Reject becaues halted in a reject state.
- Reject because of a missing transition.
- Reject bedcause the computation terminated for excessive computation steps.

The method is_exception classes the first two rejects as correct computations,
and the last as an exception.

For the purposes of the problem set, an exception is always incorrect.

- __Your machine must halt in a reasonable amount of steps.__



### Exercise A: Recognize &dashv; w &amp; w with two tapes

Write a two tape TM program to accept the language,

<code>
  { v&amp;w |  w, v in {0,1}*, and v == w }
</code>

The input will be presented as 

<code>
    &dashv; w &amp; v _ ...
</code>

<code>
    &dashv; _ ...
</code>

with the heads at the start of their respective tapes.

In phase one, copy w to the second tape. You should check that "sufficient" of the
second tape is blank so that the input of of the correct form. Phase one concludes
with the tape configuration,

<code>
    &dashv; w [&amp;] v _ ...
</code>

<code>
    &dashv; [&amp;] w _ ...
</code>

In phase two, sweep both heads right checking for symbol by symbol equality, 
until simultaneously a blank is encountered.

__Note:__ It is not possible to check the input format entirely. We can check 
whether a fininte number of tape cells are consecutively blank. While you can 
assume the format is correct, in practice, as long as every fresh cell ever
encountered is blank, we cannot disprove that the tape is entirely blank.


In [2]:
mt_m1 = """# accept w&v, w==v on a two tape machine

start: q0
accept: Z
reject: Y
tapes: 2

state: q0
    : : : & n r q1

state: q1
    0 : : 0 r r q1
    1 : : 1 r r q1
    & : : _ r l q2

state: q2
    : : : : n l q2
    : & : : n r q3

state: q3
    0 0 : : r r q3
    1 1 : : r r q3
    _ _ : : n n Z

"""

mt_m1_basic_test = (['&','0&0','1&1','01&01'],['0&1','1&0','1&11','000&0'])
create_and_test_turing_machine('exercise A basic test', mt_m1, mt_m1_basic_test, verbose='none')



Testing exercise A basic test
accept:	|&|
accept:	|0&0|
accept:	|1&1|
accept:	|01&01|
reject:	|0&1|
reject:	|1&0|
reject:	|1&11|
reject:	|000&0|
correct: 8, incorrect: 0, exceptions: 0


True

### Exercise B: Adding in binary

On a two tape machine, write a program that adds binary numbers.

The TM will have two numbers written on its tape, and will replace 
the two numbers wit their sum, and halt in accept, 
when the input format is correct.

If the input format is not correct, the TM will halt with reject, and the
tape configuration is ignored.

A binary number is written with the symbols 0 and 1. Traditionally, the least
significant bit of a binary is on the right. The number representation for this
project will write the number in reverse, with the least significant bit on the 
left. 

An empty string is not a number. The number zero must be a string of zeros with
at least one zero. A number can have additional "leading" zeros, although in our
case those leading zeros will be additional zeros on the right.

A negative number has the special minus symbol immediately to the right of the 
string of 0's and 1's. A positive number has a blank immediately to the right
of the string of 0's and 1's. 


The input will be written on the tapes as,

<code>
    &dashv; [&amp;] &lt;n1&gt; &amp; &lt;n2&gt; _ ...
</code>
<code>
    &dashv; [_] _ ...
</code>

where  &lt;n1&gt; and &lt;n2&gt; are the representations (0,1 strings) of the two
non-negative integer values n1 and n2, written as proposed. The computation
results in the tape configuration,

<code>
    &dashv; [&amp;] &lt;n3&gt; _ ...
</code>
<code>
    &dashv; [_] _ ...
</code>

where &lt;n3&gt; is the representation of the integer n1+n2.

__Example:__

Input for 2 + 6 (tape 1),

<code>
    &dashv; [&amp;] 01 &amp; 011 _ ...
</code>

results in accept with output 8 (tape 1),

<code>
    &dashv; [&amp;] 0001 _ ...
</code>

__Hints:__

- Copy n2 to tape two, ampersand and all. Then erase it from tape 1. After
this phase, the tapes contain,

<code>
    &dashv; [&amp;] &lt;n1&gt; _ ...    
</code>
<code>
    &dashv; [&amp;] &lt;n2&gt; _ ...    
</code>

- If n1 and n2 are not the same length (as 0-1 strings) add leading zeros
  to the shorter one, to make them the same length.
  

In [3]:
mt_add = """# &n1&n2 -> &n1+n2

start: q0
accept: Z
reject: Y
tapes: 2

state: q0
    & : : & r r q1

state: q1
    : : : : r n q1
    _ : : : n n Y
    & : _ : r n q2

state: q2
    0 : _ 0 r r q2
    1 : _ 1 r r q2
    & : : : n n Y
    : : _ _ l l q3

state: q3
    & & : : r r q4
    & : : : n l q3
    : & : : l n q3
    : : : : l l q3

state: q4
    0 0 : : r r q4
    0 1 : : r r q4
    1 0 : : r r q4
    1 1 : : r r q4
    _ : 0 : r r q4
    : _ : 0 r r q4
    _ _ : : l l q5

state: q5
    & & : : r r q6
    : : : : l l q5

state: q6
    _ : : : n n Y
    : _ : : n n Y
    : : : : n n q_no_carry

state: q_no_carry
    0 0 0 : r r q_no_carry
    0 1 1 : r r q_no_carry
    1 0 1 : r r q_no_carry
    1 1 0 : r r q_carry
    _ _ : : n n Z

state: q_carry
    0 0 1 : r r q_no_carry
    0 1 0 : r r q_carry
    1 0 0 : r r q_carry
    1 1 1 : r r q_carry
    _ _ 1 : r n Z

"""


mt_add_test = (
    [('&0&0','&0'),('&1&01','&11'),('&11&11','&011'),('&011&01','&0001')],
    ['','0','&','0&','&&','&0&0&']
)
basic_test_mt_add(mt_add, mt_add_test, verbose='none')



input:	|&0&0|
output:	|&0___|
** correctly calculated

input:	|&1&01|
output:	|&11___|
** correctly calculated

input:	|&11&11|
output:	|&011___|
** correctly calculated

input:	|&011&01|
output:	|&0001___|
** correctly calculated

input:	||
** correctly rejected

input:	|0|
** correctly rejected

input:	|&|
** correctly rejected

input:	|0&|
** correctly rejected

input:	|&&|
** correctly rejected

input:	|&0&0&|
** correctly rejected

correct: 10, incorrect: 0, exceptions: 0


True

### Exercise C: Subtracting in Binary

On a two tape machine, write a program that subtracts binary numbers.

The TM will have two numbers written on its tape, and will replace 
the two numbers wit their sum, and halt in accept, 
when the input format is correct.

If the input format is not correct, the TM will halt with reject, and the
tape configuration is ignored.

Neither of the two input numbers will be negative, but the result can be negative. 
The number representation was given in the discussion above, including the representation of a negative number.

__Example:__ Subtract 11 from 6. Since 11=0b1101, and 6=0b110, written in reverse,

<code>
    &dashv; [&amp;] 011 &amp; 1011 _ ...   
</code>

The result -5, written in binary, in reverse with a sign marker, 

<code>
    &dashv; [&amp;] 101-
</code>


#### Two's complement

The secret of 2's complement to represent negative numbers is to know that there are not negative or positive numbers at all. It is only how the numbers are interpreted.

For n bits, let N=2<sup>n</sup>. As an unsigned integer, n bits can represent the
integers 0 through N-1. 

__Example:__ For 3 bits, N=8. the 8 numbers are,

<code>
    0=0b000, 1=0b001, 2=0b010, 3=0b011, 4=0b100, 5=0b101, 6=0b110, 7=0b111.
</code>

As a signed integer, the numbers count up from 0 to N/2-1, and up from -N/2 to 
zero,

<code>
    -4=0b100, -3=0b101, -2=0b110, -1=0b111, 0=0b000, 1=0b001, 2=0b010, 3=0b011.
</code>

Notice,

- All negative numbers have a 1 as the most significant bit. 
- The positive numbers look like their own value; the negative numbers do not.

To make a negative number y look like -x, find the x that satisfies the equation,

<code>
    y + x = 0
</code>

because this x is allowed to be written as -y. A tricky way to find x is complement
all the bits in y then add one. This will result in a number x which is guarenteed 
to satisfy the above equation.

__Example:__ Three bits is not enough to represent 5 when signed. The largest positive number in n bits is 2<sup>n</sup>/2-1. For 3 bits the largest is 3, for 4 bits it is 7. We shall use 4 bits. Counting backwards, 

<code>
    0=0b0000, -1=0b1111, -2=0b1110, -3=0b1101, -4=0b1100, -5=0b1011
</code>

But 0b1011 lookes like 11, not -5. But 11+5=0 mod 16, so 11 is the correct
value. To get it into the form -x, take the two's complement, 

<code>
    0b1011 == complement ==> 0b0100 == add 1 ==> 0b0101
</code>

Which is binary 5.

__Hints:__ The hints for addition are good, except consider what the carry 
out needs to do. Consider the subtraction 5 minus 0. If only three bit arithmatic is used 
the result 5 can be confused with -3.



In [4]:
mt_subtract = """#  &n1&n2 -> &n3 

start: q0
accept: Z
reject: Y
tapes: 2

state: q0
    & : : & r r q1

state: q1
    : : : : r n q1
    _ : : : n n Y
    & : _ : r n q2

state: q2
    0 : _ 0 r r q2
    1 : _ 1 r r q2
    & : : : n n Y
    : : _ _ l l q3

state: q3
    & & : : r r q4
    & : : : n l q3
    : & : : l n q3
    : : : : l l q3

state: q4
    0 0 : : r r q4
    0 1 : : r r q4
    1 0 : : r r q4
    1 1 : : r r q4
    _ : 0 : r r q4
    : _ : 0 r r q4
    _ _ : : l l q5

state: q5
    & & : : r r q6
    : : : : l l q5

state: q6
    _ : : : n n Y
    : _ : : n n Y
    : : : : n n q_1s_compliment

state: q_1s_compliment
    : 0 : 1 n r q_1s_compliment
    : 1 : 0 n r q_1s_compliment
    : : : : n l q7

state: q7
    : & : : n r q_no_carry
    : : : : n l q7

state: q_no_carry
    0 0 0 : r r q_no_carry
    0 1 1 : r r q_no_carry
    1 0 1 : r r q_no_carry
    1 1 0 : r r q_carry
    _ _ : : n n q9

state: q_carry
    0 0 1 : r r q_no_carry
    0 1 0 : r r q_carry
    1 0 0 : r r q_carry
    1 1 1 : r r q_carry
    _ _ : : r n q8

state: q8
    & : : : r n q_add_1_to_result
    : : : : l n q8

state: q9
    & : : : r n q_1s_compliment_result
    : : : : l n q9

state: q_add_1_to_result
    0 : 1 : r n q10
    1 : 0 : r n q_add_1_to_result
    _ : 1 : r n q10

state: q_1s_compliment_result
    0 : 1 : r n q_1s_compliment_result
    1 : 0 : r n q_1s_compliment_result
    : : : : l n q_remove_leading_zero_and_negate

state: q10
    _ : : : l n q_remove_leading_zero
    : : : : r n q10

state: q_remove_leading_zero
    0 : _ : l n q_remove_leading_zero
    : : : : n n Z

state: q_remove_leading_zero_and_negate
    0 : _ : l n q_remove_leading_zero_and_negate
    & : : : r n q_put_zero
    : : : : r n q_negate

state: q_put_zero
    : : 0 : n n Z

state: q_negate
    : : - : n n Z

"""


mt_subtract_test = (
    [('&0&0','&0'),('&11&1','&01'),('&11&11','&0'),('&1&01','&1-')],
    ['&','0&','&1','&&']
)
basic_test_mt_add(mt_subtract, mt_subtract_test, verbose='none')



input:	|&0&0|
output:	|&0___|
** correctly calculated

input:	|&11&1|
output:	|&01___|
** correctly calculated

input:	|&11&11|
output:	|&0_____|
** correctly calculated

input:	|&1&01|
output:	|&1-___|
** correctly calculated

input:	|&|
** correctly rejected

input:	|0&|
** correctly rejected

input:	|&1|
** correctly rejected

input:	|&&|
** correctly rejected

correct: 8, incorrect: 0, exceptions: 0


True

### Extra Credit D: Fibonacci numbers

From the tape, 

<code>
    &dashv; &amp; &lt;n&gt; &amp; &lt;a_0&gt; &amp; &lt;a_1&gt;
</code>

compute the n-th fibonacci series, beginning with the a0 and a1 given, and leave
the tape in the condition,

<code>
    &dashv; &amp; &lt;a_n&gt; 
</code>

Do this on a two-tape TM.

The number representation is as described above.

__Example:__

Input:

<code>
    &dashv; [&amp;] 011 &amp; 1 &amp; 1 _ ...
</code>

Output:

<code>
    &dashv; [&amp;] 1011 _ ...
</code>

__Partial Credit:__ Partial credit for the following preliminary program, which is
a crucial half-way point to the completed extra credit program,

<code>
    assert: input is of the form &amp;(0|1)+&amp;(0|1)+&amp;(0|1)+
    while i>0:
        print tape
        decrement i
</code>

Use capitalization of the action letter to "print" the tape.

__Note:__ The default step-limit is a quadraic in the input size.
Is this sufficent? 

What is the correct step limit for your program that ensures
it has be allowed just enough steps to complete the computation?



In [1]:
mt_fibonacci = """#  &i&a0&a1 -> &ai 

start: q_sanity_check
accept: Z
reject: Y
tapes: 2

state: q_sanity_check
    & : : : r n q_sanity_check_1

state: q_sanity_check_1
    0 : : : n n q_sanity_check_2
    1 : : : n n q_sanity_check_2

state: q_sanity_check_2
    : : : : r n q_sanity_check_2
    & : : : r n q_sanity_check_3

state: q_sanity_check_3
    0 : : : n n q_sanity_check_4
    1 : : : n n q_sanity_check_4

state: q_sanity_check_4
    : : : : r n q_sanity_check_4
    & : : : r n q_sanity_check_5

state: q_sanity_check_5
    0 : : : n n q_sanity_check_6
    1 : : : n n q_sanity_check_6

state: q_sanity_check_6
    : : : : r n q_sanity_check_6
    _ : : : l n q_sanity_check_7

state: q_sanity_check_7
    : : : : l n q_sanity_check_7
    & : : : l n q_sanity_check_8

state: q_sanity_check_8
    : : : : l n q_sanity_check_8
    & : : : l n q_sanity_check_9

state: q_sanity_check_9
    : : : : l n q_sanity_check_9
    & : : : n n q_check

state: q0
    & : : : r n q1

state: q1
    & : : : r n q2
    : : : : r n q1

state: q2
    & : : & r r q3
    : : : : r n q2

state: q3
    0 : : 0 r r q3
    1 : : 1 r r q3
    _ : : _ l n q4

state: q4
    : : : : l n q4
    & : : : l n q5

state: q5
    : : : : l n q5
    & : : : n n q_add_0

######### Add Module #########

state: q_add_0
    & : : & r r q_add_1

state: q_add_1
    : : : : r n q_add_1
    & : _ : r n q_add_2

state: q_add_2
    0 : _ 0 r r q_add_2
    1 : _ 1 r r q_add_2
    : : _ _ l l q_add_3

state: q_add_3
    & & : : r r q_add_4
    & : : : n l q_add_3
    : & : : l n q_add_3
    : : : : l l q_add_3

state: q_add_4
    0 0 : : r r q_add_4
    0 1 : : r r q_add_4
    1 0 : : r r q_add_4
    1 1 : : r r q_add_4
    _ : 0 : r r q_add_4
    : _ : 0 r r q_add_4
    _ _ : : l l q_add_5

state: q_add_5
    & & : : r r q_add_6
    : : : : l l q_add_5

state: q_add_6
    : : : : n n q_add_no_carry

state: q_add_no_carry
    0 0 0 : r r q_add_no_carry
    0 1 1 : r r q_add_no_carry
    1 0 1 : r r q_add_no_carry
    1 1 0 : r r q_add_carry
    _ _ : : l l q_add_7

state: q_add_carry
    0 0 1 : r r q_add_no_carry
    0 1 0 : r r q_add_carry
    1 0 0 : r r q_add_carry
    1 1 1 : r r q_add_carry
    _ _ 1 : n l q_add_7

state: q_add_7
    : : : _ n l q_add_7
    : & : : n r q_add_8

state: q_add_8
    : : : : l n q_add_8
    & : : : r n q_add_9

state: q_add_9
    0 : _ 0 r r q_add_9
    1 : _ 1 r r q_add_9
    _ : : _ n n q_add_10

state: q_add_10
    _ : _ : l n q_add_10
    & : : : r n q_add_11

state: q_add_11
    : : 1 : n n q_add_12

state: q_add_12
    : : : : l n q_add_12
    & : : : l n q_add_13

state: q_add_13
    : : : : l n q_add_13
    & : : : n n q_sub_0

##############################

######### Sub Module #########

state: q_sub_0
    & : : & r r q_sub_1

state: q_sub_1
    : : : : r n q_sub_1
    & : _ : r n q_sub_2

state: q_sub_2
    0 : _ 0 r r q_sub_2
    1 : _ 1 r r q_sub_2
    : : _ _ l l q_sub_3

state: q_sub_3
    & & : : r r q_sub_4
    & : : : n l q_sub_3
    : & : : l n q_sub_3
    : : : : l l q_sub_3

state: q_sub_4
    0 0 : : r r q_sub_4
    0 1 : : r r q_sub_4
    1 0 : : r r q_sub_4
    1 1 : : r r q_sub_4
    _ : 0 : r r q_sub_4
    : _ : 0 r r q_sub_4
    _ _ : : l l q_sub_5

state: q_sub_5
    & & : : r r q_sub_6
    : : : : l l q_sub_5

state: q_sub_6
    : : : : n n q_sub_1s_compliment

state: q_sub_1s_compliment
    : 0 : 1 n r q_sub_1s_compliment
    : 1 : 0 n r q_sub_1s_compliment
    : : : : n l q_sub_7

state: q_sub_7
    : & : : n r q_sub_no_carry
    : : : : n l q_sub_7

state: q_sub_no_carry
    0 0 0 : r r q_sub_no_carry
    0 1 1 : r r q_sub_no_carry
    1 0 1 : r r q_sub_no_carry
    1 1 0 : r r q_sub_carry
    _ _ : : n n q_sub_17

state: q_sub_17
    & : : : r n q_sub_1s_compliment_result
    : : : : l n q_sub_17

state: q_sub_1s_compliment_result
    0 : 1 : r n q_sub_1s_compliment_result
    1 : 0 : r n q_sub_1s_compliment_result
    : : : : l n q_sub_remove_leading_zero

state: q_sub_carry
    0 0 1 : r r q_sub_no_carry
    0 1 0 : r r q_sub_carry
    1 0 0 : r r q_sub_carry
    1 1 1 : r r q_sub_carry
    _ _ : : r n q_sub_8

state: q_sub_8
    & : : : r n q_sub_add_1_to_result
    : : : : l n q_sub_8

state: q_sub_add_1_to_result
    0 : 1 : r n q_sub_9
    1 : 0 : r n q_sub_add_1_to_result
    _ : 1 : r n q_sub_9

state: q_sub_9
    _ : : : l n q_sub_remove_leading_zero
    : : : : r n q_sub_9

state: q_sub_remove_leading_zero
    0 : _ : l n q_sub_remove_leading_zero
    & : : : r n q_sub_put_zero
    : : : : r n q_sub_10

state: q_sub_put_zero
    : : 0 : r n q_sub_10

state: q_sub_10
    : : : _ n l q_sub_10
    : & : _ n l q_sub_11

state: q_sub_11
    : : : : n l q_sub_11
    : & : : n l q_sub_12

state: q_sub_12
    : : : : n l q_sub_12
    : & : : n n q_sub_13

state: q_sub_13
    : & & _ r r q_sub_13
    : 0 0 _ r r q_sub_13
    : 1 1 _ r r q_sub_13
    : _ _ : l n q_sub_14

state: q_sub_14
    : : : : l n q_sub_14
    & : : : l n q_sub_15

state: q_sub_15
    : : : : l n q_sub_15
    & : : : l n q_sub_16

state: q_sub_16
    : : : : l n q_sub_16
    & : : : n n q_check

##############################

state: q_check
    & : : : r n q_check_condition

state: q_check_condition
    0 : : : r n q_check_condition
    1 : : : n n q_not_done
    & : : : l n q_clean_left

state: q_not_done
    : : : : l n q_not_done
    & : : : n n q0

state: q_clean_left
    : : _ : l n q_clean_left
    & : : : r n q_skip_left

state: q_skip_left
    _ : : : r n q_skip_left
    & : : : r n q_skip_middle

state: q_skip_middle
    : : : : r n q_skip_middle
    & : _ : r n q_clean_right

state: q_clean_right
    : : _ : r n q_clean_right
    _ : : : l n q_skip_right

state: q_skip_right
    _ : : : l n q_skip_right
    : : : : l n q_skip_right
    & : _ & r r q_clean

state: q_clean
    0 : _ 0 r r q_clean
    1 : _ 1 r r q_clean
    _ : : _ n n q_reset_tape_2

state: q_reset_tape_2
    : : : : n l q_reset_tape_2
    : & : _ n r q_reset_tape_1

state: q_reset_tape_1
    : : : : l n q_reset_tape_1
    & : : : r n q_copy

state: q_copy
    : 0 0 _ r r q_copy
    : 1 1 _ r r q_copy
    : _ _ : r r Z

"""


mt_fibonacci_test = (
    [('&0&1&1','&1'),('&1&1&1','&1'),('&01&1&1','&01'),('&11&1&1','&11'),
    ('&001&1&1','&101'),('&101&1&1','&0001'),('&011&1&1','&1011'),],
    ['','&','0','1','1&','&0&','&1&&','&0&1&','&0&&1','&&1&1']
)
basic_test_mt_add(mt_fibonacci, mt_fibonacci_test, verbose='none')



NameError: name 'basic_test_mt_add' is not defined

### End of assigment